# Set tapered GR MFD

In [ ]:
%%html
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
<p style="font-size:60%;"><a href="javascript:code_toggle()">[Toggle Code]</a></p>

In [ ]:
%matplotlib inline
import os
import re
import sys
import h5py
import numpy
from rtree import index

import matplotlib.pyplot as plt

from openquake.mbt.oqt_project import OQtProject

from openquake.hazardlib.mfd import EvenlyDiscretizedMFD
from openquake.mbt.tools.mfd import get_moment_from_mfd
from openquake.mbt.tools.mfd import mag_to_mo, mo_to_mag
from openquake.mbt.tools.mfd import TaperedGrMFD

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]

## Set MFD

In [ ]:
m_min = float(model.m_min)
bin_width = float(model.mfd_binwidth)

opt = 1

fig1 = plt.figure(num=1, figsize=(10, 8))
fig2 = plt.figure(num=2, figsize=(10, 8))

for key in model.sources:
    source = model.sources[key]
    if re.match('AreaSource', source.source_type) and 'mo_mcs' in source.__dict__:
        
        # Compute the 
        m_c = mo_to_mag(source.mo_mcs)
        
        if opt == 0:
            mags = numpy.arange(m_min, source.mmax_expected+bin_width/2, bin_width)
        elif opt == 1:
            mags = numpy.arange(m_min, m_c+3.0, bin_width)
        else:
            raise ValueError('Unknown option')
        #
        # The first parameter is the "truncation moment" 
        gr_trunc = TaperedGrMFD(mag_to_mo(0.), source.mo_mcs, source.b_gr)
        mo = mag_to_mo(numpy.array(mags))   
        ccdfA = gr_trunc.get_ccdf(mo)*10.**(source.a_gr)
        aa = ccdfA[:-1]-ccdfA[1:]
        
        # Filter our bins with rates lower than threshold_rate
        if opt == 1:
            idx = numpy.nonzero(aa > 1e-6)
            aa = aa[idx]
        
        # Set the new MFD
        source.mfd = EvenlyDiscretizedMFD(m_min+bin_width/2, bin_width, list(aa))
        source.mo_from_mfd = get_moment_from_mfd(source.mfd)
        
        plt.figure(1)
        plt.plot(mags[idx], aa)
        plt.figure(2)
        plt.plot(source.mo_strain, source.mo_from_mfd, 'o')
        
        print("%s\t%6.3e\t%6.3e\t%6.3e\t%6.3e\t%6.3e\t%5.3f" % (source.source_id, source.a_gr, 
                                                         source.b_gr, m_c, source.mo_strain,
                                                         source.mo_from_mfd,
                                                         source.mo_from_mfd/source.mo_strain))
plt.figure(1)
plt.semilogy()
plt.figure(2)
plt.grid(which='both')
plt.xlim([1e18, 3e19])
plt.ylim([1e18, 3e19])

oqtkp.save()